In [1]:
import findspark
findspark.init()

In [ ]:
sc.stop()

In [2]:
from pyspark import SparkContext
sc=SparkContext(master="local", appName="first app" )

## 맛보기

In [3]:
# Action - collect()
rdd = sc.parallelize(range(1,11))
result = rdd.collect()
print(result) # [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
id(result)    #139931150042440

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


140691153533384

In [4]:
# Action - count()
rdd = sc.parallelize(range(1, 11))
result = rdd.count()
print(result)  # 10
id(result)     # 10914784

10


10914784

## Map과 관련된 연산들

In [5]:
# transformation Action - map, collect()
rdd1 = sc.parallelize(range(1, 6))
rdd2= rdd1.map(lambda v : v+1)
print(rdd2.collect())
# [2, 3, 4, 5, 6]

[2, 3, 4, 5, 6]


In [6]:
# map 
rdd1 = sc.parallelize(["apple,orange", "grape,apple,mango", "blueberry,tomato,orange"])
rdd2 = rdd1.map(lambda s: s.split(","))
print(rdd2.collect())
# [['apple', 'orange'], ['grape', 'apple', 'mango'], ['blueberry', 'tomato', 'orange']]

[['apple', 'orange'], ['grape', 'apple', 'mango'], ['blueberry', 'tomato', 'orange']]


In [7]:
# flatMap
rdd1 = sc.parallelize(["apple,orange", "grape,apple,mango", "blueberry,tomato,orange"])
rdd2 = rdd1.flatMap(lambda s: s.split(","))
print(rdd2.collect())
# ['apple', 'orange', 'grape', 'apple', 'mango', 'blueberry', 'tomato', 'orange']

['apple', 'orange', 'grape', 'apple', 'mango', 'blueberry', 'tomato', 'orange']


In [72]:
# python코드 없이 만들어봄  # page 111
# flatMap return type TraversableOnce[U]   # None 리턴되는 것이 중요
rdd1 = sc.parallelize(["apple,orange", "grape,apple.mango", "blueberry,tomato,orange"])
def deflog(log):
    print(str(log)) # 실행되지 않음
    if "apple" == str(log):
        return list[log]
    else:
        return list(), print(type(log))
# deflog = [ if "apple" == str(log) ]
rdd2 = rdd1.flatMap(deflog)  # transformation에서는 error발생치 않음
print(rdd2)                  # PythonRDD[12] at RDD at PythonRDD.scala:53
print(rdd2.collect())       #  [[], None, [], None, [], None]

PythonRDD[275] at RDD at PythonRDD.scala:53
[[], None, [], None, [], None]


In [9]:
# mapPartitions
# map()과 flatMap() RDD의 각 요소를 하나씩 처리
# mapPartition() 파티션단위로 처리:파티션에 속한 모든 요소의 컬렉션에 대한 이터레이터(Iterator)를 입력으로 사용,리턴도 이터레이터
# 파티션 단위의 중간산출물을 만들거나 DB 연결과 같은 고비용의 자원을 파티션 단위로 공유해 사용할 수 있다는 장점

# increase
def increase(numbers):
    print("DB 연결 !!!")
    return(i + 1 for i in numbers)

#rdd1 = sc.parallelize(range(1,11))   # 옵션으로 partition 정할 수 있다
rdd1 = sc.parallelize(range(1,11), 3)   # 옵션으로 partition 정할 수 있다
rdd2 = rdd1.mapPartitions(increase)    # 함수자체를 매개변수로
print(rdd2.collect())
#[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [10]:
# mapPartitionsWithIndex
# 파티션에 속한 요소의 정보 + 해당파티션의 인덱스 함께 전달
#increaseWithIndex
def increaseWithIndex(idx, numbers):
    print("partitions !!!")
    for i in numbers:
        if(idx == 1):
            yield i + 1  # yield 일시정지 변수기억 next()

            
rdd1 = sc.parallelize(range(1,11), 3)
rdd2 = rdd1.mapPartitionsWithIndex(increaseWithIndex)  # 함수자체를 매개변수로
print(rdd2.collect())
# [5, 6, 7]  <== [ 4, 5, 6] 파티션이 index 1로 전달되었음을 추정할 수 있다

[5, 6, 7]


In [11]:
# mapValues  # RDD의 요소가 key value의 쌍으로 이루는 경우 페어RDD(PairRDD)
# 인자로 받은 value에 해당하는 요소에만 적용, 그 결과로 구성된 새로운 RDD 생성

rdd1 = sc.parallelize(["a", "b", "c"])

# PairRDD 생성
rdd2 = rdd1.map(lambda v : (v, 1))

rdd3 = rdd2.mapValues(lambda i: i + 1)
print(rdd3.collect())
# [('a', 2), ('b', 2), ('c', 2)]

[('a', 2), ('b', 2), ('c', 2)]


In [12]:
# flatMapValues

rdd1 = sc.parallelize([(1, "a,b"), (2, "a,c"), (1, "d,e")])

# PairRDD 생성
rdd2 = rdd1.flatMapValues(lambda s: s.split(","))
print(rdd2.collect())
# [(1, 'a'), (1, 'b'), (2, 'a'), (2, 'c'), (1, 'd'), (1, 'e')]

[(1, 'a'), (1, 'b'), (2, 'a'), (2, 'c'), (1, 'd'), (1, 'e')]


## 그룹과 관련된 연산들

In [13]:
# zip  # key value # 요소의 개수 같아야 한다
rdd1 = sc.parallelize(["a", "b", "c"])
rdd2 = sc.parallelize([1, 2, 3])

result = rdd1.zip(rdd2)
print(result) #org.apache.spark.api.java.JavaPairRDD@37a7333b
print(result.collect())
# [('a', 1), ('b', 2), ('c', 3)]

org.apache.spark.api.java.JavaPairRDD@4e0d2262
[('a', 1), ('b', 2), ('c', 3)]


In [14]:
# zipPartitions  파티션단위로 zip()연산 수행 # 파티션 개수 같아야 한다
# 파이썬에서는 사용할 수 없음


In [15]:
# groupBy # value값을 새로운 key값으로 하는 (key, value그룹(시퀸스))으로 생성
# 인자로 전달하는 함수가 각 그룹의 키를 결정하는 역할 담당

rdd1 = sc.parallelize(range(1, 11))
rdd2 = rdd1.groupBy(lambda v: "even" if v%2==0 else "odd")
print(rdd2) # PythonRDD[38] at RDD at PythonRDD.scala:53
print(rdd2.collect())
#[('odd', <pyspark.resultiterable.ResultIterable object at 0x7f444277e860>), ('even', <pyspark.resultiterable.ResultIterable object at 0x7f444277e828>)]
for x in rdd2.collect():
    #print(x[0], x[1])
# odd <pyspark.resultiterable.ResultIterable object at 0x7f44432759e8>
# even <pyspark.resultiterable.ResultIterable object at 0x7f44604d7160>
    print(x[0], list(x[1]))
# odd [1, 3, 5, 7, 9]
# even [2, 4, 6, 8, 10]

PythonRDD[28] at RDD at PythonRDD.scala:53
[('odd', <pyspark.resultiterable.ResultIterable object at 0x7ff5363a9b00>), ('even', <pyspark.resultiterable.ResultIterable object at 0x7ff5363a9ba8>)]
odd [1, 3, 5, 7, 9]
even [2, 4, 6, 8, 10]


In [16]:
# groupByKey  # RDD구성요소 key, value에서만 가능
# key기준으로 같은 키를 가진 요소들로 그룹을 만들고
rdd1 = sc.parallelize(["a", "b", "c", "b", "c"]).map(lambda v: (v, 1))
rdd2 = rdd1.groupByKey()
for x in rdd2.collect():
    print(x[0], list(x[1]))
# a [1]
# b [1, 1]
# c [1, 1]

a [1]
b [1, 1]
c [1, 1]


In [17]:
# cogroup   ## RDD구성요소 key, value에서만 가능
# join과 비교
# 여러 RDD에서 같은 키를 갖는 값 요소를 찾아서 키와 그 키에 속하는 요소의 시퀀스(List, Vector 등의 상위클래스인 Iterable)
# 구성된 튜플, 그 튜플로 구성된 RDD 생성 
# [ Tuple(key, Tuple(rdd1요소들의 집합, rdd2요소들의 집합)), ... ]
rdd1 = sc.parallelize([("k1", "v1"), ("k2", "v2"), ("k1", "v3")])
rdd2 = sc.parallelize([("k1", "v4")])
result = rdd1.cogroup(rdd2)
print(result) # PythonRDD[120] at RDD at PythonRDD.scala:53
print(result.collect())
# [('k2', (<pyspark.resultiterable.ResultIterable object at 0x7f4440be7e48>, <pyspark.resultiterable.ResultIterable object at 0x7f4440be7c50>)), ('k1', (<pyspark.resultiterable.ResultIterable object at 0x7f4440be7dd8>, <pyspark.resultiterable.ResultIterable object at 0x7f4440be4f28>))]
for x in result.collect():
    print(x[0], list(x[1][0]), list(x[1][1]))
# k2 ['v2'] []
# k1 ['v1', 'v3'] ['v4']

PythonRDD[44] at RDD at PythonRDD.scala:53
[('k2', (<pyspark.resultiterable.ResultIterable object at 0x7ff53636dba8>, <pyspark.resultiterable.ResultIterable object at 0x7ff53636dc88>)), ('k1', (<pyspark.resultiterable.ResultIterable object at 0x7ff53636db70>, <pyspark.resultiterable.ResultIterable object at 0x7ff53636dda0>))]
k2 ['v2'] []
k1 ['v1', 'v3'] ['v4']


### 집합과 관련된 연산들

In [18]:
# distinct  # RDD의 원소에서 중복을 제외한 요소로만 구성된 새로운 RDD 생성
rdd = sc.parallelize([1,2,3,1,2,3,1,2,3])
result = rdd.distinct()
print(result.collect())
#[1, 2, 3]

[1, 2, 3]


In [19]:
# cartesian  # RDD구성요소 key, value에서만 가능
rdd1 = sc.parallelize([1,2,3])
rdd2 = sc.parallelize(["a", "b", "c"])
result = rdd1.cartesian(rdd2)
print(result.collect())
#[(1, 'a'), (1, 'b'), (1, 'c'), (2, 'a'), (2, 'b'), (2, 'c'), (3, 'a'), (3, 'b'), (3, 'c')]

[(1, 'a'), (1, 'b'), (1, 'c'), (2, 'a'), (2, 'b'), (2, 'c'), (3, 'a'), (3, 'b'), (3, 'c')]


In [20]:
# subtract  # 두 개의 RDD가 있을 때 rdd1- rdd2
rdd1 = sc.parallelize(["a", "b", "c", "d", "e"])
rdd2 = sc.parallelize(["d", "e"])
result = rdd1.subtract(rdd2)
print(result.collect())
#['b', 'c', 'a']

['b', 'c', 'a']


In [21]:
# union 
rdd1 = sc.parallelize(["a", "b", "c"])
rdd2 = sc.parallelize(["d", "e", "f"])
result = rdd1.union(rdd2)
print(result.collect())
#['a', 'b', 'c', 'd', 'e', 'f']

['a', 'b', 'c', 'd', 'e', 'f']


In [22]:
# intersection  #  중복제외됨
rdd1 = sc.parallelize (["a", "a", "b", "c" ])
rdd2 = sc.parallelize (["a", "a", "c", "c"])
result = rdd1.intersection(rdd2)
print(result.collect())
# ['c', 'a']

['c', 'a']


In [23]:
# join  # RDD구성요소 key, value에서만 가능
# cogroup와 비교  # Action값을 바로 볼 수 있다
# [ Tuple(key, Tuple(rdd1요소들의 집합, rdd2요소들의 집합)), ... ]
rdd1 = sc.parallelize (["a", "b", "c", "d", "e"]).map(lambda v: (v,1))
rdd2 = sc.parallelize (["b", "c"]).map(lambda v: (v, 2))
result = rdd1.join(rdd2)
print(result.collect())
#[('b', (1, 2)), ('c', (1, 2))]

[('b', (1, 2)), ('c', (1, 2))]


In [24]:
# leftOuterJoin, rightOuterJoin  # RDD구성요소 key, value에서만 가능
# key기준으로 외부조인을 수행
rdd1 = sc.parallelize (["a", "b", "c", "d", "e"]).map(lambda v: (v,1))
rdd2 = sc.parallelize (["b", "c"]).map(lambda v: (v, 2))
result1 = rdd1.leftOuterJoin(rdd2)
result2 = rdd1.rightOuterJoin(rdd2)
print(f'leftOuterJoin: {result1.collect()}')
print(f'rightOuterJoin: {result2.collect()}')
# leftOuterJoin: [('b', (1, 2)), ('c', (1, 2)), ('d', (1, None)), ('a', (1, None)), ('e', (1, None))]
# rightOuterJoin: [('b', (1, 2)), ('c', (1, 2))]

leftOuterJoin: [('b', (1, 2)), ('c', (1, 2)), ('d', (1, None)), ('a', (1, None)), ('e', (1, None))]
rightOuterJoin: [('b', (1, 2)), ('c', (1, 2))]


In [25]:
# subtractByKey  # RDD구성요소 key, value에서만 가능
rdd1 = sc.parallelize (["a", "b"]).map(lambda v: (v,1))
rdd2 = sc.parallelize (["b"]).map(lambda v: (v, 1))
result = rdd1.subtractByKey(rdd2)
print(result.collect())
# [('a', 1)]

[('a', 1)]


## 집계와 관련된 연산들

In [26]:
# reduceByKey  # RDD구성요소 key, value에서만 가능
# 2개의 값을 하나로 합치는 함수를 인자로 전달받는데, 이 함수는 결합, 교환법칙 성립
rdd = sc.parallelize (["a", "b", "b"]).map(lambda v: (v,1))
result = rdd.reduceByKey(lambda v1, v2: v1 + v2)
print(result.collect())
# [('a', 1), ('b', 2)]

[('a', 1), ('b', 2)]


In [27]:
# foldByKey   # RDD구성요소 key, value에서만 가능
# reduceByKey와는 달리 병합연산의 초기값을 메서드의 인자로 전달해서 병합시 사용
# ex) 더하는 함수 0, 두 문자열 연결 공백문자""를 초기값으로 사용가능
# 하지만 이 때 초기값이 반복해도 연산결과에 영향을 주지 않는 값이여야 함
# 함수는 교환법칙은 만족안해도 되고 결합법칙은 만족해야 함

rdd = sc.parallelize (["a", "b", "b"]).map(lambda v: (v,1))
result = rdd.foldByKey(0, lambda v1,v2:v1+v2)
print(result.collect())
# [('a', 1), ('b', 2)]

[('a', 1), ('b', 2)]


In [28]:
# error  # page 137
# combineByKey    # RDD구성요소 key, value에서만 가능
class Record:
        
    def __init__(self, amount, number=1):
        self.amount = amount
        self.number = number
        
    def addAmt(self, amount):
        return Record(self.amount + amount, self.number + 1)
    
    def __add__(self, other):
        amount = self.amount + other.amount
        number = self.number + other.number 
        return Record(amount, number)
        
    def __str__(self):
        return "avg:" + str(self.amount / self.number)

    def __repr__(self):
        return 'Record(%r, %r)' % (self.amount, self.number)
        
# combineBy
def createCombiner(v):
    return Record(v)

# combineBy
def mergeValue(c, v):
    return c.addAmt(v)

# combineBy
def mergeCombiners(c1, c2):
    return c1 + c2

rdd = sc.parallelize([("Math", 100), ("Eng", 80), ("Math", 50), ("Eng", 70), ("Eng", 90)])
result = rdd.combineByKey(lambda v: createCombiner(v), lambda c, v: mergeValue(c, v), lambda c1, c2: mergeCombiners(c1, c2))
print('Math', result.collectAsMap()['Math'], 'Eng', result.collectAsMap()['Eng'])
#print(str(result.collectAsMap()))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 40.0 failed 1 times, most recent failure: Lost task 0.0 in stage 40.0 (TID 58, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/pyspark/pkg/spark-2.4.5-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/home/pyspark/pkg/spark-2.4.5-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/pyspark/pkg/spark-2.4.5-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 142, in dump_stream
    for obj in iterator:
  File "/home/pyspark/pkg/spark-2.4.5-bin-hadoop2.7/python/pyspark/rdd.py", line 1796, in add_shuffle_key
    yield outputSerializer.dumps(items)
  File "/home/pyspa

In [ ]:
# aggregateByKey   # RDD구성요소 key, value에서만 가능
# combineByKey()와 동일하나 초기값 생성하는 부분만 다름
# result = rdd.aggregateByKey(zero)(mergeValue, mergecombiners)

## pipe 및 파티션과 관련된 연산

In [29]:
# pipe
rdd = sc.parallelize( ["1,2,3", "4,5,6", "7,8,9"])
result = rdd.pipe("cut -f 1,3 -d ,") # -field  1,3출력 구분자 -d=delimeter default tab, 여기서는 ,
print(result.collect())
# ['1,3', '4,6', '7,9']

['1,3', '4,6', '7,9']


In [30]:
# 최초 설정한 partition 수 조정필요시
# coalesce     : 줄이기만 가능         셔플 수행하라는 옵션지정하지 않는  셔플을 사용하지 않기 때문
# repartition : 늘리거나 줄일 수 있음  셔플 기반으로 동작
rdd1= sc.parallelize(list(range(1, 11)),10)
rdd2 = rdd1.coalesce(5)
rdd3 = rdd2.repartition(10)
print(f'partition size: {rdd1.getNumPartitions()}')
print(f'partition size: {rdd2.getNumPartitions()}')
print(f'partition size: {rdd3.getNumPartitions()}')

partition size: 10
partition size: 5
partition size: 10


In [98]:
# page 142
# repartitionAndSortWithinPartitions    # RDD구성요소 key, value에서만 가능
# RDD를 구성하는 모든 데이터를 특정 기준에 따라 여러 개의 파티션으로 분리, 각 파티션단위로 정렬수행한 뒤, 
# 그 결과로 새로운  RDD을  생성
# 메서드 실행시 각 데이터가 어떤 파티션에 속할지 결정하기 위한 파티셔너(Partitioner)를 설정
# 파티셔너는 각 데이터의 키 값을 이용해 데이터가 속할 파티션 결정, 키 값을 이용한 정렬도 함께 수행
# 즉 파티션 재할당을 위해 셔플을 수행하는 단계에서 정렬도 함께 다루게 되어 파티션과 정렬 따로하는 것보다 높은 성능

import random
data = [random.randrange(1,100) for i in range(0, 10)]
#print(data) # [23, 99, 61, 18, 84, 58, 75, 13, 55, 47]

rdd1 = sc.parallelize(data).map(lambda v: (v, "-"))
#print(rdd1.collect())  # [(73, '-'), (79, '-'), (47, '-'), (98, '-'), (33, '-'), (51, '-'), (43, '-'), (68, '-'), (95, '-'), (69, '-')]

rdd2 = rdd1.repartitionAndSortWithinPartitions(3, lambda x: x) # HashPartitioner 파티셔너 3
# print(rdd2) # PythonRDD[187] at RDD at PythonRDD.scala:53
# rdd2.count # <bound method RDD.count of PythonRDD[315] at RDD at PythonRDD.scala:53>

# 아래코드 에러는 없으나 결과 출력물이 안나옴  # cli에서는 나옴 (foreach관련) # foreach(함수)
# rdd2.foreachPartition(lambda values: print(list(values))) # RDD의 파티션단위로 특정함수를 실행
# [Stage 0:> (0 + 1) / 1][(6, '-'), (42, '-'), (42, '-'), (90, '-')]
# [(49, '-'), (52, '-'), (58, '-')]
# [(35, '-'), (53, '-'), (89, '-')]

rdd2.foreachPartition(  lambda values: print( list( map(lambda v: v, values)) ) ) 
# [(6, '-'), (42, '-'), (42, '-'), (90, '-')]
# [(49, '-'), (52, '-'), (58, '-')]
# [(35, '-'), (53, '-'), (89, '-')]


#아래코드 에러는 없으나 결과 출력물이 안나옴   # cli 에서는 나옴 (foreach)
# def test(it):
#     print("="*5)
#     print ( list( map(lambda x: x, it) ) )
# rdd2.foreachPartition(test) 
# =====
# [(6, '-'), (42, '-'), (42, '-'), (90, '-')]
# =====
# [(49, '-'), (52, '-'), (58, '-')]
# =====
# [(35, '-'), (53, '-'), (89, '-')]


# 아래코드는 Py4JJavaErro # cli에서도 error # foreach 2번
# rdd2.foreachPartition( lambda it: it.foreach(lambda v: print(v))  ) # RDD의 파티션단위로 특정함수를 실행

# 아래코드는 Py4JJavaErro # error
# def test(it):
#     print("="*5)
#     it.foreach( lambda x: print(x) )             
# rdd2.foreachPartition(test)  



In [48]:
# partitionBy   # RDD구성요소 key, value에서만 가능 
# 파티셔너(Partitioner) 인자로 전달
# HashPartitioner   : 각 요소의 키 값으로부터 해시값을 취해 이 값을 기준으로 파티션을 결정하는 방법
# Rangepartitioner : 순서가 있는 요소들(Sortable)로 구성된 RDD에 사용가능,
#                    각 요소를 목표 파티션 크기에 맞게 일정크기의 구간으로 나누는 방식

rdd1 = sc.parallelize( [ ("apple", 1), ("mouse", 1), ("monitor", 1)],  5 )
rdd2 = rdd1.partitionBy(3)
print(f'rdd1: {rdd1.getNumPartitions()} ')  # rdd1: 5 
print(f'rdd2: {rdd2.getNumPartitions()} ')  # rdd2: 3 


rdd1: 5 
rdd2: 3 


In [92]:
# list ( map (함수, iter) ) 연습

# li = [1, 2, 3]
# result = map(lambda i: i * i, li)
# print(next(result))  # 1
# print(next(result))  # 1
# print( list(map(lambda i: i * i, li)) )  #[1, 4, 9]


## 필터와 정렬 연산

In [93]:
# filter   # python filter # list( filter(함수,iter) )
rdd1 = sc.parallelize(range(1,6))
rdd2 = rdd1.filter(lambda i : i > 2)
print(rdd2.collect())
# [3, 4, 5]

[3, 4, 5]


In [94]:
# sortByKey    # RDD구성요소 key, value에서만 가능
# 키 값을 기준으로 요소를 정렬하는 연산  # 소팅 완료 후 파티션 내부의 요소는 소팅 순서상 인접한요소로 재구성
rdd = sc.parallelize( [("q",1), ("z", 1), ("a", 1)])
result = rdd.sortByKey()
print(result.collect())
# [('a', 1), ('q', 1), ('z', 1)]

[('a', 1), ('q', 1), ('z', 1)]


In [96]:
# keys()  키에 해당하는 요소로 구성된 RDD 생성
# values() 값에 해당하는 요소로 구성된 RDD 생성
rdd = sc.parallelize([("k1", "v1"), ("k2", "v2"), (" k3", "v3")])
result1 = rdd.keys()
result2 = rdd.values()
print(result1.collect()) # ['k1', 'k2', ' k3']
print(result2.collect()) # ['v1', 'v2', 'v3']

['k1', 'k2', ' k3']
['v1', 'v2', 'v3']


In [ ]:
# sample
# 스칼라 API    sample(withReplacement: Boolean, fraction: Double, seed: Long=Utils.nextLong): RDD[t]
# withReplacement : 복원추출 수행여부 True, False 
# fraction 복원일경우 : 샘플내에서 각 요소가 나타나는 횟수에 대한 기대값, 즉 각요소의 평균발생 횟수, 0이상
#          비복원일경우:          각 요소가 샘플에 포함될 확률, 0 ~ 1 사이 값으로 지정가능

# 샘플의 크기를 정해놓고 추출을 실행하지 않음  fraction 0.5 전체크기 50% 샘플추출아님
# 크기를 정해놓은 메소드 takeSample()
# seed : 일반적인 무작위 값 추출시 사용하는 것과 유사한 개념으로 반복시행시 결과가 바뀌지 않도록 제어 목적

rdd = sc.parallelize(range(1, 101))
result1 = rdd.sample(False, 0.5, 666) 
result2 = rdd.sample(True, 1.5, 777)
print(f'result1.count(): {result1.count()}  result1.take(5): {result1.take(5)}') 
# result1.count(): 54  result1.take(5): [2, 4, 5, 6, 7]
print(f'result2.count(): {result2.count()}  result2.take(5): {result2.take(5)}') 
# result2.count(): 153  result2.take(5): [1, 4, 4, 4, 4]